## Import

In [ ]:
%env CUDA_VISIBLE_DEVICES=2,3
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
import torch

In [ ]:
# DEVICE = torch.device('cpu')
DEVICE = torch.device('cuda')
DEVICE

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
%run ../datasets/iu_xray.py
%run ../training/report_generation/hierarchical.py
%run ../utils/nlp.py

In [ ]:
train_dataset = IUXRayDataset('train')
len(train_dataset)

## Compute dataset stats

In [ ]:
from collections import Counter

In [ ]:
def get_dataset_stats(dataset):
    word_appearances = dict()

    words_count = []
    sentences_count = []

    for image, report in tqdm(dataset):
        # Save appearances
        for word in report:
            if word not in word_appearances:
                word_appearances[word] = 0
            word_appearances[word] += 1

        # Must end with a dot
        if report[-1] != END_OF_SENTENCE_IDX:
            report.append(END_OF_SENTENCE_IDX)

        # Count words
        n_words = len(report)
        words_count.append(n_words)

        # Count sentences
        n_sentences = report.count(END_OF_SENTENCE_IDX)
        sentences_count.append(n_sentences)

    return word_appearances, words_count, sentences_count

In [ ]:
word_appearances, words_count, sentences_count = get_dataset_stats(train_dataset)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_counter(counter, title):
    x = list(counter.keys())
    y = list(counter.values())
    
    plt.bar(x, y)
    plt.title(title)

In [ ]:
plot_counter(Counter(words_count), 'words')

In [ ]:
plot_counter(Counter(sentences_count), 'sentences')

In [ ]:
counter = Counter(sentences_count)

In [ ]:
def get_percentiles(counter):
    items = sorted(counter.items())

    keys, values = zip(*items)

    return list(zip(keys, np.cumsum(values) / total * 100))

In [ ]:
get_percentiles(Counter(words_count))

In [ ]:
get_percentiles(Counter(sentences_count))

## Evaluate models in subsets

In [ ]:
%run -n train_report_generation.py
%run datasets/__init__.py
%run models/checkpoint/__init__.py
%run training/report_generation/flat.py
%run training/report_generation/hierarchical.py
%run models/report_generation/__init__.py

In [ ]:
def eval_in_subset(run_name, compiled_model, debug=True, max_n_words=None, max_n_sentences=None,
                   device='cuda'):
    # Create datasets
    vocab = compiled_model.metadata['vocab']
    train_dataset = IUXRayDataset('train', vocab=vocab)
    val_dataset = IUXRayDataset('val', vocab=vocab)
    test_dataset = IUXRayDataset('test', vocab=vocab)
    
    # Prepare subsets
    subset_kwargs = {
        'max_n_words': max_n_words,
        'max_n_sentences': max_n_sentences,
    }
    
    train_subset = create_report_dataset_subset(train_dataset, **subset_kwargs)
    val_subset = create_report_dataset_subset(val_dataset, **subset_kwargs)
    test_subset = create_report_dataset_subset(test_dataset, **subset_kwargs)
    
    # Decide hierachical
    decoder_name = compiled_model.metadata['decoder_kwargs']['decoder_name']
    hierarchical = is_decoder_hierarchical(decoder_name)
    if hierarchical:
        create_dataloader = create_hierarchical_dataloader
    else:
        create_dataloader = create_flat_dataloader

    # Create dataloaders
    BS = 50
    train_dataloader = create_dataloader(train_subset, batch_size=BS)
    val_dataloader = create_dataloader(val_subset, batch_size=BS)
    test_dataloader = create_dataloader(test_subset, batch_size=BS)
    
    # Create a suffix
    if max_n_words:
        suffix = f'max-words-{max_n_words}'
    elif max_n_sentences:
        suffix = f'max-sentences-{max_n_sentences}'
        
    evaluate_and_save(run_name,
                      compiled_model.model,
                      train_dataloader,
                      val_dataloader,
                      test_dataloader,
                      hierarchical=hierarchical,
                      debug=debug,
                      device=device,
                      suffix=suffix,
                     )

In [ ]:
eval_n_words = [
    20 , # --> 15%
    25 , # --> 26%
    27 , # --> 33%
    33 , # --> 50%
#     39 , # --> 66%
#     41 , # --> 70%
    44 , # --> 75%
#     47 , # --> 80%
#     58 , # --> 90%
    # None, # --> 100%
]

In [ ]:
eval_n_sentences = [
#     1, # 1.2324835387472564
#     2, # 4.761100793516799
    3, # 25.730204288367382
    4, # 55.10720918453487
    5, # 76.66722944453824
    6, # 89.39726489954415
#     7, # 95.03629917271653
#     8, # 97.6194496032416
#     9, # 98.86881647813608
#     10, # 99.42596657099443
#     11, # 99.71298328549722
#     12, # 99.89869998311667
#     13, # 99.96623332770555
#     17, # 99.98311666385278
#     18, # 100
]

In [ ]:
run_names = [
#     '0717_041434_lstm_lr0.0001_densenet-121',
    '0716_211601_lstm-att_lr0.0001_densenet-121', # faltan 33 y 34
#     '0717_015057_h-lstm_lr0.0001_densenet-121',
#     '0716_234501_h-lstm-att_lr0.0001_densenet-121',
]
debug = False

In [ ]:
for run_name in run_names:
    compiled_model = load_compiled_model_report_generation(run_name,
                                                           debug=debug,
                                                           multiple_gpu=True,
                                                           device=DEVICE)
    for n_words in tqdm(eval_n_words):
        eval_in_subset(run_name,
                       compiled_model,
                       max_n_words=n_words,
                       max_n_sentences=None,
                       debug=debug,
                       device=DEVICE,
                      )
    for n_sentences in tqdm(eval_n_sentences):
        eval_in_subset(run_name,
                       compiled_model,
                       max_n_words=None,
                       max_n_sentences=n_sentences,
                       debug=debug,
                       device=DEVICE,
                      )

## Eval H-LSTM outputs

### Load model

In [ ]:
%run ../models/report_generation/__init__.py
%run ../models/checkpoint/__init__.py

In [ ]:
# run_name = '0716_234501_h-lstm-att_lr0.0001_densenet-121'
# run_name = '0717_015057_h-lstm_lr0.0001_densenet-121'
# run_name = '0720_192858_lstm_lr0.0001_densenet-121_size256'
# run_name = '0717_041434_lstm_lr0.0001_densenet-121'
run_name = '0716_211601_lstm-att_lr0.0001_densenet-121'
debug = False

In [ ]:
compiled_model = load_compiled_model_report_generation(run_name,
                                                       debug=debug,
                                                       device=DEVICE,
                                                      )

_ = compiled_model.model.eval()

In [ ]:
compiled_model.metadata['decoder_kwargs']

In [ ]:
VOCAB = compiled_model.metadata['vocab']
len(VOCAB)

### Load data

In [ ]:
%run ../datasets/iu_xray.py

In [ ]:
dataset_kwargs = {
    'max_samples': None,
    'frontal_only': False,
    'image_size': (512, 512),
    'vocab': VOCAB,
}

train_dataset = IUXRayDataset(dataset_type='train', **dataset_kwargs)
val_dataset = IUXRayDataset(dataset_type='val', **dataset_kwargs)
test_dataset = IUXRayDataset(dataset_type='test', **dataset_kwargs)
len(train_dataset), len(val_dataset), len(test_dataset)

### Eval

In [ ]:
from ignite.engine import Engine, Events

In [ ]:
%run ../training/report_generation/hierarchical.py
%run ../training/report_generation/flat.py
%run ../utils/nlp.py
%run ../utils/__init__.py
%run ../metrics/report_generation/__init__.py

In [ ]:
is_hierarchical = compiled_model.metadata['decoder_kwargs']['decoder_name'].startswith('h-')

if is_hierarchical:
    get_step_fn = get_step_fn_hierarchical
    create_dataloader = create_hierarchical_dataloader
else:
    get_step_fn = get_step_fn_flat
    create_dataloader = create_flat_dataloader

In [ ]:
dataloader = create_dataloader(test_dataset, batch_size=1)

In [ ]:
free = True

In [ ]:
tester = Engine(get_step_fn(compiled_model.model, training=False, free=free, device=DEVICE))
attach_metrics_report_generation(tester, hierarchical=is_hierarchical, free=free)
attach_report_writer(tester, VOCAB, run_name, debug=True)

In [ ]:
tester.state.dataloader.dataset.dataset_type

In [ ]:
tester.run(dataloader, 1)

In [ ]:
tester.state.metrics

In [ ]:
def eval_sample(compiled_model, image, report,
                show=True, device=DEVICE, free=False, **kwargs):
    report_reader = ReportReader(compiled_model.metadata['vocab'])
    
    is_hierarchical = compiled_model.metadata['decoder_kwargs']['decoder_name'].startswith('h-')
    
    # Prepare inputs
    images = image.unsqueeze(0).to(device)
    if is_hierarchical:
        reports = split_sentences_and_pad(report)
    else:
        reports = torch.tensor(report)

    reports = reports.unsqueeze(0).to(device)
    
    # Pass thru model
    if not is_hierarchical:
        del kwargs['max_sentences']
    tup = compiled_model.model(images, reports, free=free, **kwargs)
    
    # Parse outputs
    if is_hierarchical:
        generated = _flatten_gen_reports(tup[0], tup[1])
    else:
        generated = tup[0]
        _, generated = generated.max(dim=-1)

    generated = generated.squeeze(0).cpu()
    
    # Print result
    original_report = report_reader.idx_to_text(report)
    generated_report = report_reader.idx_to_text(generated)
    if show:
        print(original_report)
        print('-'*20)
        print(generated_report)
        
    return original_report, generated_report

In [ ]:
idx = 200

image, report = train_dataset[idx]

gt, gen = eval_sample(compiled_model, image, report,
                      free=True, max_sentences=100, max_words=100)

### Eval metrics on dataset

#### BLEU

In [ ]:
from pycocoevalcap.bleu import bleu_scorer

In [ ]:
s = bleu_scorer.BleuScorer(n=4)
s += (gen, [gt])
scores_avg, scores_all = s.compute_score()
scores_avg, len(scores_all[0])

In [ ]:
def eval_dataset(dataset, max_samples=None, free=False):
    scorer = bleu_scorer.BleuScorer(n=4)
    report_lens = []
    
    n_samples = len(dataset) if max_samples is None else max_samples
    for idx in tqdm(range(n_samples)):
        image, report = dataset[idx]

        report_lens.append(len(report))

        gt, gen = eval_sample(compiled_model, image, report, show=False,
                              free=free, max_sentences=100, max_words=20)

        scorer += (gen, [gt])
    scores_avg, scores_all = scorer.compute_score()
    scores_all = np.array(scores_all)

    return scores_avg, scores_all, report_lens

In [ ]:
train_results = eval_dataset(train_dataset, free=True)
train_results[0]

In [ ]:
val_results = eval_dataset(val_dataset, free=True)
val_results[0]

In [ ]:
test_results = eval_dataset(test_dataset, free=True)
test_results[0]

#### Rouge-L

In [ ]:
from pycocoevalcap.rouge import rouge

In [ ]:
idx = 800

image, report = train_dataset[idx]

gt, gen = eval_sample(compiled_model, image, report,
                      free=True, max_sentences=100, max_words=100)

scorer = rouge.Rouge()
scorer.calc_score([gen], [gt])

In [ ]:
def eval_dataset(dataset, max_samples=None, free=False):
    scorer = rouge.Rouge()
    scores = []
    report_lens = []
    
    n_samples = len(dataset) if max_samples is None else max_samples
    for idx in tqdm(range(n_samples)):
        image, report = dataset[idx]

        report_lens.append(len(report))

        gt, gen = eval_sample(compiled_model, image, report, show=False,
                              free=free, max_sentences=100, max_words=20)

        scores.append(scorer.calc_score([gen], [gt]))

    return np.mean(scores), np.array(scores), report_lens

In [ ]:
test_results = eval_dataset(test_dataset, free=True)
test_results[0]

#### CIDEr

In [ ]:
from pycocoevalcap.cider import cider_scorer

In [ ]:
idx = 800
image, report = train_dataset[idx]

gt, gen = eval_sample(compiled_model, image, report,
                      free=True, max_sentences=100, max_words=100)

scorer = cider_scorer.CiderScorer(n=4)
scorer += (gen, [gt])

idx = 0
image, report = train_dataset[idx]

gt, gen = eval_sample(compiled_model, image, report,
                      free=True, max_sentences=100, max_words=100)
scorer += (gen, [gt])

scorer.compute_score()

In [ ]:
def eval_dataset(dataset, max_samples=None, free=False):
    scorer = cider_scorer.CiderScorer(n=4)
    report_lens = []
    
    n_samples = len(dataset) if max_samples is None else max_samples
    for idx in tqdm(range(n_samples)):
        image, report = dataset[idx]

        report_lens.append(len(report))

        gt, gen = eval_sample(compiled_model, image, report, show=False,
                              free=free, max_sentences=100, max_words=20)

        scorer += (gen, [gt])
    scores_avg, scores_all = scorer.compute_score()
    scores_all = np.array(scores_all)

    return scores_avg, scores_all, report_lens

In [ ]:
val_results = eval_dataset(val_dataset, free=False)
val_results[0]

In [ ]:
test_results = eval_dataset(test_dataset, free=False)
test_results[0]

#### Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
results = test_results

In [ ]:
all_scores = results[1]
# all_scores = all_scores.mean(axis=0)
# all_scores = all_scores[0, :]
all_scores.shape

In [ ]:
lens = np.array(results[2])
lens.shape

###### Plot scatter

In [ ]:
plt.scatter(lens, all_scores)
plt.xlabel('Report len')
plt.ylabel('Metric')

###### Adjust linear regression

In [ ]:
from sklearn.linear_model import LinearRegression as LR

In [ ]:
lr = LR()
lr.fit(lens.reshape(-1, 1), all_scores)

In [ ]:
lr.coef_, lr.intercept_

###### Plot hist

In [ ]:
plt.hist(all_scores, bins=50)
plt.xlabel('Metric')
plt.ylabel('Occurences')

In [ ]:
results[0]

In [ ]:
results[1].mean(axis=0).shape

#### Eval sample with ordered scores

In [ ]:
results = test_results
dataset = test_dataset

In [ ]:
scores_all = results[1]
# BLEU
# ordered_values = [
#     (i, *scores_all[:, i]) 
#     for i in range(scores_all.shape[1])
# ]
ordered_values = list(enumerate(scores_all)) # ROUGE-L, CIDEr
len(ordered_values)

In [ ]:
ordered_values = sorted(ordered_values, key=lambda x:x[1], reverse=True)

In [ ]:
idx, *scores = ordered_values[2]

print(scores)
image, report = dataset[idx]

gt, gen = eval_sample(compiled_model, image, report,
                      free=True, max_sentences=100, max_words=100)

## Debug metrics

### Distinct words

In [ ]:
%run ../metrics/report_generation/distinct_words.py

In [ ]:
dw = DistinctWords()
dw.reset()

In [ ]:
reports = torch.tensor([[1, 2, 3, 4],
                        [21, 12, 1, 0],
                       ])

In [ ]:
reports2 = reports + 1
reports2

In [ ]:
dw.update(reports)
dw.update(reports2)
dw.compute()

In [ ]:
dw.words_seen

### Distinct sentences

In [ ]:
%run ../metrics/report_generation/distinct_sentences.py

In [ ]:
ds = DistinctSentences()
ds.reset()

In [ ]:
reports = torch.tensor([
    [[11, 21, 37, 4],
     [29, 32, 52, 4],
    ],
    [[11, 21, 37, 4],
     [29, 32, 52, 4],
    ],
    [[12, 23, 47, 4],
     [30, 33, 53, 5],
    ],
])

In [ ]:
ds.update(reports)
ds.update(reports + 1)
ds.compute()

In [ ]:
ds.sentences_seen

## Debug others

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np

In [ ]:
a = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 1, 1],
             ])
b = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 1, 1],
             ])

In [ ]:
roc_auc_score(a, b, average=None)

In [ ]:
import hashlib

In [ ]:
%run -n ../eval_report_generation_chexpert_labeler.py

In [ ]:
run_name = '0915_172446_dummy-common-sentences-100'

In [ ]:
evaluate_run(run_name, debug=True, max_samples=30, override=True)

## Debug MIRQI

In [ ]:
import pandas as pd
import re

In [ ]:
%run -n ../eval_report_generation_mirqi.py

### Load MIRQI output

In [ ]:
df = pd.read_csv('~/software/MIRQI/testing2.csv')
df.fillna('', inplace=True)
df.head()

In [ ]:
attributes_gt = _attributes_to_list(df['attributes-gt'])
attributes_gen = _attributes_to_list(df['attributes-gen'])
len(attributes_gt), len(attributes_gen)

In [ ]:
df['attributes-gt']

In [ ]:
%run -n ../eval_report_generation_mirqi.py

In [ ]:
scores = MIRQI_v2(attributes_gt, attributes_gen)

In [ ]:
scores['MIRQI-v2-attr-p']

In [ ]:
idx = 2
attributes_gt[idx], attributes_gen[idx]

## MIRQI Examples

In [ ]:
%run -n ../eval_report_generation_mirqi.py
# %run -n ~/software/MIRQI/evaluate.py

### MIRQI original def

In [ ]:
def MIRQI(gt_list, cand_list, pos_weight=0.8, attribute_weight=0.3, verbose=False):
    """Compute the score of matching keyword and associated attributes between gt list and candidate list.
       It returns two scores:   MIRQI-r (recall: hits in gt)
                                MIRQI-p (precision: correct ratio of all candidates)
    """

    MIRQI_r = []
    MIRQI_p = []
    MIRQI_f = []

    for gt_report_entry, cand_report_entry in zip(gt_list, cand_list):
        attribute_cand_all = []

        pos_count_in_gt = 0
        pos_count_in_cand = 0
        tp = 0.0
        fp = 0.0
        tn = 0.0
        fn = 0.0

        for gt_entity in gt_report_entry:
            if gt_entity[2] == 'NEGATIVE':
                continue
            pos_count_in_gt = pos_count_in_gt + 1
        neg_count_in_gt = len(gt_report_entry) - pos_count_in_gt

        for entity_index, cand_entity in enumerate(cand_report_entry):
            if cand_entity[2] == 'NEGATIVE':
                for entity_index, gt_entity in enumerate(gt_report_entry):
                    if  gt_entity[1] == cand_entity[1]:
                        if gt_entity[2] == 'NEGATIVE':
                            tn = tn + 1     # true negative hits
                            break
                        else:
                            fn = fn + 1     # false negative hits
                            break
            else:
                pos_count_in_cand = pos_count_in_cand + 1
                for entity_index, gt_entity in enumerate(gt_report_entry):
                    if gt_entity[1] == cand_entity[1]:
                        if gt_entity[2] == 'NEGATIVE':
                            fp = fp + 1     # false positive hits
                            break
                        else:
                            tp = tp + 1.0 - attribute_weight    # true positive hits (key words part)
                            # count attribute hits
                            if gt_entity[3] == '':
                                break
                            attributes_all_gt = gt_entity[3].split('/')
                            attribute_hit_count = 0
                            for attribute in attributes_all_gt:
                                if attribute in cand_entity[3]:
                                    attribute_hit_count = attribute_hit_count + 1
                            # true positive hits (attributes part)
                            temp = attribute_hit_count/len(attributes_all_gt)*attribute_weight
                            tp = tp + temp
                            break
        neg_count_in_cand = len(cand_report_entry) - pos_count_in_cand
        #
        # calculate score for positive/uncertain mentions
        if pos_count_in_gt == 0 and pos_count_in_cand == 0:
            score_r = 1.0
            score_p = 1.0
        elif pos_count_in_gt == 0 and pos_count_in_cand != 0:
            score_r = 0.0
            score_p = 0.0
        elif pos_count_in_gt != 0 and pos_count_in_cand == 0:
            score_r = 0.0
            score_p = 0.0
        else:
            score_r = tp / (tp + fn + 0.000001)
            score_p = tp / (tp + fp + 0.000001)

        # calculate score for negative mentions
        # if neg_count_in_cand != 0 and neg_count_in_gt != 0:
        if tn != 0:
            score_r = score_r * pos_weight + tn / (tn + fp + 0.000001) * (1.0 - pos_weight)
            score_p = score_p * pos_weight + tn / (tn + fn + 0.000001) * (1.0 - pos_weight)

        MIRQI_r.append(score_r)
        MIRQI_p.append(score_p)
        rec_prec = (score_r + score_p)
        MIRQI_f.append(2*(score_r * score_p) / rec_prec if rec_prec != 0.0 else 0.0)

    scores = {
        'MIRQI-r': MIRQI_r,
        'MIRQI-p': MIRQI_p,
        'MIRQI-f': MIRQI_f,
    }

    return scores

### Robust matching

#### Repeated nodes with different attributes

In [ ]:
report_gt = "right effusion with mild atelectasis. left effusion is also present."
entities_gt = [
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'right/present'],
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'left/pleural'],
]
report_gen = report_gt
entities_gen = [
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'left/pleural'],
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'right/present'],
]
{
    **MIRQI([entities_gt], [entities_gen]),
    **MIRQI_v2([entities_gt], [entities_gen]),
}

#### GT nodes matched twice

In [ ]:
report_gt = "right pleural effusion."
entities_gt = [
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'right'],
]
report_gen = "right pleural effusion. left pleural effusion"
entities_gen = [
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'right'],
    ['effusion', 'Pleural Effusion', 'POSITIVE', 'left'],
]
{
    **MIRQI([entities_gt], [entities_gen]),
    **MIRQI_v2([entities_gt], [entities_gen]),
}